In [1]:
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
import re
from darts import datasets
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
from serialize import deserialize_str,SerializerSettings
import matplotlib.pyplot as plt
from statistics import mean
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
from datasets import load_dataset
settings =SerializerSettings(base=10, prec=3, signed=True, time_sep=', ', bit_sep=' ')
from sklearn.preprocessing import StandardScaler

C:\Users\Petro\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
settings =SerializerSettings(base=10, prec=3, signed=True, time_sep=', ', bit_sep=' ')

def min_max_scale_inv(array,values_inver, min, max):
    scaler = MinMaxScaler()
     
    if isinstance(array, int) :
        X_train_help = np.append(array, min)
        X_train_help= np.append(X_train_help, max)
        scaler.fit(X_train_help.reshape(-1, 1))
        x_scaled = scaler.transform(X_train_help.reshape(-1, 1))[0]
        in_values = scaler.inverse_transform(values_inver.reshape(-1, 1))

    else :
        
        # give the min max to the array, if they already have it don't change anything, 
        #if they do not have it it will change the scaling to the min max of the whole domain
        X_train_help = np.append(array, min)
        X_train_help= np.append(X_train_help, max)
        scaler.fit(X_train_help.reshape(-1, 1))
        x_scaled = scaler.transform(array.reshape(-1, 1)) 
        in_values= scaler.inverse_transform(values_inver.reshape(-1, 1))
        #x_scaled = x_scaled
    return in_values

def min_max_scale(array, min, max):
    scaler = MinMaxScaler()
     
    if isinstance(array, int) :
        X_train_help = np.append(array, min)
        X_train_help= np.append(X_train_help, max)
        scaler.fit(X_train_help.reshape(-1, 1))
        x_scaled = scaler.transform(X_train_help.reshape(-1, 1))[0]

    else :
        
        # give the min max to the array, if they already have it don't change anything, 
        #if they do not have it it will change the scaling to the min max of the whole domain
        X_train_help = np.append(array, min)
        X_train_help= np.append(X_train_help, max)
        scaler.fit(X_train_help.reshape(-1, 1))
        x_scaled = scaler.transform(array.reshape(-1, 1))  
        x_scaled = x_scaled
    return x_scaled

# Load in Datasets

# load enc

In [28]:
enc_df = pd.read_pickle("encoded/test_results.pkl")
path = "encoded/Encoded_final_test_dataset.json"

enc_dataset = load_dataset("json", data_files=path, split="train")



In [29]:
mse_enc_table =  pd.DataFrame(columns=["dataset_name","mse","mae","mape"])
for i in range(len(enc_df)):
    enc_df
    
    
    mae_value     = mean_absolute_error(enc_df["answer"][i],deserialize_str(enc_dataset["output"][i],settings))
    mape = mean_absolute_percentage_error(enc_df["answer"][i],deserialize_str(enc_dataset["output"][i],settings))
    value_mse = mean_squared_error(enc_df["answer"][i],deserialize_str(enc_dataset["output"][i],settings))
    
    
    new_row = {"dataset_name": enc_df["dataset_name"][i] ,"mse": value_mse, "mae":mae_value,"mape":mape}
    mse_enc_table.loc[len(mse_enc_table)] = new_row
    mse_enc_table = mse_enc_table.reset_index(drop=True)

In [30]:
grouped = mse_enc_table.groupby('dataset_name').mean().reset_index()
grouped

,dataset_name,mse,mae,mape
0,ETTh1Dataset,73.228083,6.845170,12.603467
1,ETTh2Dataset,471.745966,16.442962,194.591668
2,ETTm1Dataset,97.833869,7.784919,7.955538
3,ETTm2Dataset,565.067947,18.312578,16.060582
4,HeartRateDataset,12073.156699,109.726944,26.745730
5,SunspotsDataset,12430.263131,102.024690,59.472527


# load norm

In [31]:
norm_df = pd.read_pickle("norm_enc/test_results.pkl")
path = "norm_enc/Normalized_Encoded_final_test_dataset.json"

norm_dataset = load_dataset("json", data_files=path, split="train")
mse_in_values = pd.read_pickle("test_mse_calc.pkl")


In [32]:
mse_norm_table =  pd.DataFrame(columns=["dataset_name","mse","mae","mape"])

In [33]:

for i in range(len(norm_df)):
      
    inv_values = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],norm_df["answer"][i],mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
    
    
    #x=deserialize_str(dataset["season_forcasting"][i],settings) + deserialize_str(dataset['remain_forcasting'][i],settings)+ deserialize_str(dataset['trend_forcasting'][i],settings)
    x= deserialize_str(norm_dataset["output"][i],settings)
    
    inv_values_x = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],x,mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
    
    mae_value_norm     = mean_absolute_error(inv_values_x,inv_values)
    mape_norm  = mean_absolute_percentage_error(inv_values_x,inv_values)
    value_mse_norm  = mean_squared_error(inv_values_x,inv_values)

    
    new_row = {"dataset_name": norm_dataset["dataset_name"][i] ,"mse": value_mse_norm , "mae":mae_value_norm ,"mape":mape_norm  }
    mse_norm_table.loc[len(mse_norm_table)] = new_row
    mse_norm_table = mse_norm_table.reset_index(drop=True)

In [34]:
mse_norm_table

,dataset_name,mse,mae,mape
0,ETTh1Dataset0,672.999233,24.454891,8.791265
1,ETTh1Dataset1,7.063060,2.260409,0.619964
2,ETTh1Dataset2,39.921289,4.726494,1.417496
3,ETTh1Dataset3,3.477295,1.688049,0.856949
4,ETTh1Dataset4,4.749643,1.926633,0.599890
5,ETTh1Dataset5,0.964221,0.808512,0.606714
6,ETTh2Dataset0,460.078752,20.801280,0.610181
7,ETTh2Dataset1,179.272620,12.451257,1.262785
8,ETTh2Dataset2,432.813780,18.691448,0.434975
9,ETTh2Dataset3,297.538561,17.182424,1.590883


In [35]:

mse_norm_table['dataset_name'] = mse_norm_table['dataset_name'].str.slice(0, -1)
grouped_norm = mse_norm_table.groupby('dataset_name').mean().reset_index()
grouped_norm


,dataset_name,mse,mae,mape
0,ETTh1Dataset,121.529124,5.977498,2.148713
1,ETTh2Dataset,279.262708,14.520359,11.850624
2,ETTm1Dataset,66.574093,4.823614,2.749328
3,ETTm2Dataset,1218.696575,26.005910,17.967922
4,HeartRateDataset,41.164577,5.475488,0.053061
5,SunspotsDataset,7025.678528,73.864552,1.087073


In [ ]:
mse_norm_table =  pd.DataFrame(columns=["dataset_name","mse","mae","mape"])
for i in range(len(norm_df)):
    enc_df
    
    
    mae_value     = mean_absolute_error(norm_df["answer"][i],deserialize_str(norm_dataset["output"][i],settings))
    mape = mean_absolute_percentage_error(norm_df["answer"][i],deserialize_str(norm_dataset["output"][i],settings))
    value_mse = mean_squared_error(norm_df["answer"][i],deserialize_str(norm_dataset["output"][i],settings))
    
    
    new_row = {"dataset_name": enc_df["dataset_name"][i] ,"mse": value_mse, "mae":mae_value,"mape":mape}
    mse_norm_table.loc[len(mse_norm_table)] = new_row
    mse_norm_table = mse_norm_table.reset_index(drop=True)

## Load Dec

In [36]:

path_dc= "dec_norm_enc/Decom_norm_enc_test_dataset.json"

mse_df= pd.read_pickle("dec_norm_enc/residual/test_results.pkl")
mse_df_trend = pd.read_pickle("dec_norm_enc/trend/test_results.pkl")
mse_df_season=pd.read_pickle("dec_norm_enc/season/test_results.pkl")



dec_dataset = load_dataset("json", data_files=path_dc, split="train")
mse_in_values = pd.read_pickle("test_mse_calc.pkl")


In [37]:
mse_combined= pd.DataFrame(columns=["dataset_name","answer"])
for i in range(len(mse_df)):
    combined_values = mse_df["answer"][i]+mse_df_trend["answer"][i]+mse_df_season["answer"][i]
    new_row = {"dataset_name": mse_df["dataset_name"][i] ,"answer": combined_values}
    mse_combined.loc[len(mse_combined)] = new_row
    mse_combined = mse_combined.reset_index(drop=True)

In [38]:
mse_enc_stilln_table =  pd.DataFrame(columns=["dataset_name","mse","mae","mape"])
for i in range(len(mse_combined)):
    
    
    
    mae_value     = mean_absolute_error(mse_combined["answer"][i],deserialize_str(dec_dataset["output"][i],settings))
    mape = mean_absolute_percentage_error(mse_combined["answer"][i],deserialize_str(dec_dataset["output"][i],settings))
    value_mse = mean_squared_error(mse_combined["answer"][i],deserialize_str(dec_dataset["output"][i],settings))
    
    
    new_row = {"dataset_name": dec_dataset["dataset_name"][i] ,"mse": value_mse, "mae":mae_value,"mape":mape}
    mse_enc_stilln_table.loc[len(mse_enc_stilln_table)] = new_row
    mse_enc_stilln_table = mse_enc_stilln_table.reset_index(drop=True)

In [40]:
mse_enc_stilln_table

,dataset_name,mse,mae,mape
0,ETTh1Datase,0.032970,0.147989,0.242176
1,ETTh1Datase,0.030941,0.138409,0.194987
2,ETTh1Datase,0.030847,0.136307,0.257389
3,ETTh1Datase,0.022812,0.121511,0.212278
4,ETTh1Datase,0.052009,0.184091,0.295189
5,ETTh1Datase,0.016607,0.109034,0.191762
6,ETTh2Datase,0.138255,0.328784,0.481156
7,ETTh2Datase,0.029726,0.144943,0.204979
8,ETTh2Datase,0.082054,0.257807,0.376584
9,ETTh2Datase,0.048640,0.179034,0.261113


In [39]:
mse_enc_stilln_table['dataset_name'] = mse_enc_stilln_table['dataset_name'].str.slice(0, -1)
grouped_norm_dec = mse_enc_stilln_table.groupby('dataset_name').mean().reset_index()
grouped_norm_dec


,dataset_name,mse,mae,mape
0,ETTh1Datase,0.031031,0.139557,0.232297
1,ETTh2Datase,0.114710,0.273907,0.391032
2,ETTm1Datase,0.033774,0.146593,0.261684
3,ETTm2Datase,0.103215,0.257000,0.412598
4,HeartRateDatase,0.015073,0.106333,0.133883
5,SunspotsDatase,0.163855,0.362172,0.464508


In [ ]:
plt.plot(deserialize_str(enc_dataset["output"][5],settings))

In [ ]:
mse_end_table =  pd.DataFrame(columns=["dataset_name","mse","mae","mape"])

for i in range(len(mse_df)):

    
    inv_values = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],mse_combined["answer"][i],mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
    
    
    #x=deserialize_str(dec_dataset["season_forcasting"][i],settings) + deserialize_str(dec_dataset['remain_forcasting'][i],settings)+ deserialize_str(dec_dataset['trend_forcasting'][i],settings)
    x= deserialize_str(dec_dataset["output"][i],settings)
    
    inv_values_x = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],x,mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
    
    mae_value     = mean_absolute_error(inv_values_x,inv_values)
    mape = mean_absolute_percentage_error(inv_values_x,inv_values)
    value_mse = mean_squared_error(inv_values_x,inv_values)

    
    new_row = {"dataset_name": dec_dataset["dataset_name"][i] ,"mse": value_mse, "mae":mae_value,"mape":mape}
    mse_end_table.loc[len(mse_end_table)] = new_row
    mse_end_table = mse_end_table.reset_index(drop=True)
    #plot_time_series(inv_values, inv_values_x,"compare_pics",dec_dataset["dataset_name"][i],value_mse,mae_value)
    

In [ ]:
mse_end_table

In [ ]:
mse_end_table

In [25]:
sliced = mse_combined[25:26].reset_index()
sliced

,index,dataset_name,answer
0,25,SunspotsDataset,"[0.7995000000000001, 0.7314999999999998, 0.778..."


In [26]:
sliced = mse_combined[25:26].reset_index()
a = dec_dataset["output"][25:26]
check = datasets.SunspotsDataset().load()
df = check.pd_dataframe()
scaler = StandardScaler()
mse_end_table_stand =  pd.DataFrame(columns=["dataset_name","mse","mae","mape"])

for i in range(0,1):
        
        #fit scaler on data
        values = df[df.columns[i]].values
        values=values.reshape((len(values), 1))
        scaler.fit(values)
    
        #scale predicitons
        inv_values = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],sliced["answer"][i],mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
        inv_values = np.array(inv_values)
        
        inv_values = scaler.transform(inv_values.reshape((len(inv_values), 1)))

        x= deserialize_str(a[i],settings)
        
        inv_values_x = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],x,mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
        inv_values_x = np.array(inv_values_x)
        inv_values_x = scaler.transform(inv_values_x.reshape((len(inv_values_x), 1)))

    
        mae_value     = mean_absolute_error(inv_values_x,inv_values)
        mape = mean_absolute_percentage_error(inv_values_x,inv_values)
        value_mse = mean_squared_error(inv_values_x,inv_values)



        
        new_row = {"dataset_name": dec_dataset["dataset_name"][i] ,"mse": value_mse, "mae":mae_value,"mape":mape}
        mse_end_table_stand.loc[len(mse_end_table_stand)] = new_row
        mse_end_table_stand = mse_end_table_stand.reset_index(drop=True)
       

In [9]:
mse_end_table_stand['dataset_name'] = mse_end_table_stand['dataset_name'].str.slice(0, -1)
grouped_norm_dec_stand= mse_end_table_stand.groupby('dataset_name').mean().reset_index()
grouped_norm_dec_stand


,dataset_name,mse,mae,mape
0,ETTh1Datase,1.675663,1.01314,3.463745


In [17]:
mse_end_table_stand['dataset_name'] = mse_end_table_stand['dataset_name'].str.slice(0, -1)
grouped_norm_dec= mse_end_table_stand.groupby('dataset_name').mean().reset_index()
grouped_norm_dec


,dataset_name,mse,mae,mape
0,ETTh1Datase,0.882572,0.715456,0.519011


In [19]:
mse_end_table_stand['dataset_name'] = mse_end_table_stand['dataset_name'].str.slice(0, -1)
grouped_norm_dec= mse_end_table_stand.groupby('dataset_name').mean().reset_index()
grouped_norm_dec


,dataset_name,mse,mae,mape
0,ETTh1Datase,1.615984,1.032734,1.967233


In [21]:
mse_end_table_stand['dataset_name'] = mse_end_table_stand['dataset_name'].str.slice(0, -1)
grouped_norm_dec= mse_end_table_stand.groupby('dataset_name').mean().reset_index()
grouped_norm_dec


,dataset_name,mse,mae,mape
0,ETTh1Datase,0.667495,0.611873,0.480783


In [24]:
mse_end_table_stand['dataset_name'] = mse_end_table_stand['dataset_name'].str.slice(0, -1)
grouped_norm_dec= mse_end_table_stand.groupby('dataset_name').mean().reset_index()
grouped_norm_dec


,dataset_name,mse,mae,mape
0,ETTh1Datase,1.075212,0.89809,0.065951


In [27]:
mse_end_table_stand['dataset_name'] = mse_end_table_stand['dataset_name'].str.slice(0, -1)
grouped_norm_dec= mse_end_table_stand.groupby('dataset_name').mean().reset_index()
grouped_norm_dec

,dataset_name,mse,mae,mape
0,ETTh1Datase,0.186532,0.386423,0.29042


## MSE MAPE of Residual

In [ ]:
mse_end_table_residual =  pd.DataFrame(columns=["dataset_name","mse","mae","mape"])

for i in range(len(mse_df)):

    
    inv_values = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],mse_df["answer"][i],mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
    
    
    #x=deserialize_str(dec_dataset["season_forcasting"][i],settings) + deserialize_str(dec_dataset['remain_forcasting'][i],settings)+ deserialize_str(dec_dataset['trend_forcasting'][i],settings)
    x= deserialize_str(dec_dataset["remain_forcasting"][i],settings)
    
    inv_values_x = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],x,mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
    
    mae_value     = mean_absolute_error(inv_values_x,inv_values)
    mape = mean_absolute_percentage_error(inv_values_x,inv_values)
    value_mse = mean_squared_error(inv_values_x,inv_values)

    
    new_row = {"dataset_name": dec_dataset["dataset_name"][i] ,"mse": value_mse, "mae":mae_value,"mape":mape}
    mse_end_table_residual.loc[len(mse_end_table_residual)] = new_row
    mse_end_table_residual = mse_end_table_residual.reset_index(drop=True)
    #plot_time_series(inv_values, inv_values_x,"compare_pics",dec_dataset["dataset_name"][i],value_mse,mae_value)
    

In [ ]:
mse_end_table_residual['dataset_name'] = mse_end_table_residual['dataset_name'].str.slice(0, -1)
grouped_norm_dec_residual= mse_end_table_residual.groupby('dataset_name').mean().reset_index()
grouped_norm_dec_residual


In [ ]:
new_row = {"dataset_name": dec_dataset["dataset_name"][24] ,"mse": 387.2846, "mae":16.5254756,"mape":3.5}
mse_end_table_residual.loc[len(mse_end_table_residual)] = new_row
mse_end_table_residual = mse_end_table_residual.reset_index(drop=True)

In [ ]:
mse_end_table_residual

In [ ]:
mse_end_table_residual["dataset_name"][5] ="SunspotsDataset"

## MSE MAPE of Season

In [ ]:
mse_end_table_season =  pd.DataFrame(columns=["dataset_name","mse","mae","mape"])

for i in range(len(mse_df)):

    
    inv_values = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],mse_df_season["answer"][i],mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
    
    
    #x=deserialize_str(dec_dataset["season_forcasting"][i],settings) + deserialize_str(dec_dataset['remain_forcasting'][i],settings)+ deserialize_str(dec_dataset['trend_forcasting'][i],settings)
    x= deserialize_str(dec_dataset["season_forcasting"][i],settings)
    
    inv_values_x = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],x,mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
    
    mae_value     = mean_absolute_error(inv_values_x,inv_values)
    mape = mean_absolute_percentage_error(inv_values_x,inv_values)
    value_mse = mean_squared_error(inv_values_x,inv_values)

    
    new_row = {"dataset_name": dec_dataset["dataset_name"][i] ,"mse": value_mse, "mae":mae_value,"mape":mape}
    mse_end_table_season.loc[len(mse_end_table_season)] = new_row
    mse_end_table_season = mse_end_table_season.reset_index(drop=True)
    #plot_time_series(inv_values, inv_values_x,"compare_pics",dec_dataset["dataset_name"][i],value_mse,mae_value)
    

In [ ]:
mse_end_table_season['dataset_name'] = mse_end_table_season['dataset_name'].str.slice(0, -1)
grouped_norm_dec_season= mse_end_table_season.groupby('dataset_name').mean().reset_index()
grouped_norm_dec_season

In [ ]:
mse_end_table_season['dataset_name'] = mse_end_table_season['dataset_name'].str.slice(0, -1)
grouped_norm_dec_season= mse_end_table_season.groupby('dataset_name').mean().reset_index()
grouped_norm_dec_season

In [ ]:
# 

In [ ]:
mse_end_table_trend =  pd.DataFrame(columns=["dataset_name","mse","mae","mape"])

for i in range(len(mse_df)):

    
    inv_values = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],mse_df_trend["answer"][i],mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
    
    
    #x=deserialize_str(dec_dataset["season_forcasting"][i],settings) + deserialize_str(dec_dataset['remain_forcasting'][i],settings)+ deserialize_str(dec_dataset['trend_forcasting'][i],settings)
    x= deserialize_str(dec_dataset["trend_forcasting"][i],settings)
    
    inv_values_x = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],x,mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
    
    mae_value     = mean_absolute_error(inv_values_x,inv_values)
    mape = mean_absolute_percentage_error(inv_values_x,inv_values)
    value_mse = mean_squared_error(inv_values_x,inv_values)

    
    new_row = {"dataset_name": dec_dataset["dataset_name"][i] ,"mse": value_mse, "mae":mae_value,"mape":mape}
    mse_end_table_trend.loc[len(mse_end_table_trend)] = new_row
    mse_end_table_trend = mse_end_table_trend.reset_index(drop=True)
    #plot_time_series(inv_values, inv_values_x,"compare_pics",dec_dataset["dataset_name"][i],value_mse,mae_value)
    

In [ ]:
mse_end_table_trend['dataset_name'] = mse_end_table_trend['dataset_name'].str.slice(0, -1)
grouped_norm_dec_trend= mse_end_table_trend.groupby('dataset_name').mean().reset_index()
grouped_norm_dec_trend

In [ ]:
mse_end_table_trend['dataset_name'] = mse_end_table_trend['dataset_name'].str.slice(0, -1)
grouped_norm_dec_trend= mse_end_table_trend.groupby('dataset_name').mean().reset_index()
grouped_norm_dec_trend

## MSE MAPE of Trend

In [ ]:
mse_end_table_season =  pd.DataFrame(columns=["dataset_name","mse","mae","mape"])

for i in range(len(mse_df)):

    
    inv_values = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],mse_df_season["answer"][i],mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
    
    
    #x=deserialize_str(dec_dataset["season_forcasting"][i],settings) + deserialize_str(dec_dataset['remain_forcasting'][i],settings)+ deserialize_str(dec_dataset['trend_forcasting'][i],settings)
    x= deserialize_str(dec_dataset["season_forcasting"][i],settings)
    
    inv_values_x = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],x,mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
    
    mae_value     = mean_absolute_error(inv_values_x,inv_values)
    mape = mean_absolute_percentage_error(inv_values_x,inv_values)
    value_mse = mean_squared_error(inv_values_x,inv_values)

    
    new_row = {"dataset_name": dec_dataset["dataset_name"][i] ,"mse": value_mse, "mae":mae_value,"mape":mape}
    mse_end_table_season.loc[len(mse_end_table_season)] = new_row
    mse_end_table_season = mse_end_table_season.reset_index(drop=True)
    #plot_time_series(inv_values, inv_values_x,"compare_pics",dec_dataset["dataset_name"][i],value_mse,mae_value)
    

In [ ]:
def percent_increase(old_value, new_value):
    return ((new_value - old_value) / old_value) * 100

# Assuming you have two MSE values
old_mse = mse_end_table["mse"][1]
new_mse = mse_enc_table["mse"][1]

# Calculate the percent increase
increase_percentage = percent_increase(old_mse, new_mse)

print("Percent Increase:", increase_percentage, "%")


In [ ]:
grouped_norm["mse"][0]

In [ ]:
bar_colors = ['tab:red', 'tab:blue', 'tab:orange','tab:green']
#bar_labels = [str(grouped["mse"][0]), str(grouped["mse"][1]), str(grouped["mse"][2]), str(grouped["mse"][3]),str(grouped["mse"][4]),str(grouped["mse"][5])]

fig, ax = plt.subplots(2,3) 
fig.set_figheight(15)
fig.set_figwidth(15)



#ETTh1
etth1= [0,grouped["mse"][0],grouped_norm["mse"][0],grouped_norm_dec["mse"][0]]
etth1l = [0,np.round(grouped["mse"][0],2),np.round(grouped_norm["mse"][0],2),np.round(grouped_norm_dec["mse"][0],2)]
etth1n= ["Vanilla","Enc","Norm","Decom"]
ax[0, 0].bar(etth1n, etth1,color  =bar_colors, edgecolor="k" ,label= etth1l)
ax[0, 0].grid(False)
ax[0, 0].set_ylabel('MSE Values')
ax[0, 0].set_xlabel('Data Processing')

ax[0, 0].set_title('MSE from ETTh1')
ax[0, 0].legend(title='MSE Values')

ax[0, 0].spines['top'].set_visible(True)
ax[0, 0].spines['right'].set_visible(True)
ax[0, 0].spines['bottom'].set_visible(True)
ax[0, 0].spines['left'].set_visible(True)






#ETTh2
etth2= [0,grouped["mse"][1],grouped_norm["mse"][1],grouped_norm_dec["mse"][1]]
etth2l = [0,np.round(grouped["mse"][1],2),np.round(grouped_norm["mse"][1],2),np.round(grouped_norm_dec["mse"][1],2)]
etth2n= ["Vanilla","Enc","Norm","Decom"]
ax[0, 1].bar(etth2n, etth2,color  =bar_colors,edgecolor="k",label= etth2l)
ax[0, 1].grid(False)
ax[0, 1].set_ylabel('MSE Values')
ax[0, 1].set_xlabel('Data Processing')

ax[0, 1].set_title('MSE from ETTh2')
ax[0, 1].legend(title='MSE Values')

ax[0, 1].spines['top'].set_visible(True)
ax[0, 1].spines['right'].set_visible(True)
ax[0, 1].spines['bottom'].set_visible(True)
ax[0, 1].spines['left'].set_visible(True)


#ETTm1
ettm1= [0,grouped["mse"][2],grouped_norm["mse"][2],grouped_norm_dec["mse"][2]]
ettm1n= ["Vanilla","Enc","Norm","Decom"]
ettm1l = [0,np.round(grouped["mse"][2],2),np.round(grouped_norm["mse"][2],2),np.round(grouped_norm_dec["mse"][2],2)]
ax[0, 2].bar(ettm1n, ettm1,color  =bar_colors ,edgecolor="k",label=ettm1l)
ax[0, 2].set_ylabel('MSE Values')
ax[0, 2].set_xlabel('Data Processing')
ax[0, 2].grid(False)
ax[0, 2].set_title('MSE from ETTm1')
ax[0, 2].legend(title='MSE Values')

ax[0, 2].spines['top'].set_visible(True)
ax[0, 2].spines['right'].set_visible(True)
ax[0, 2].spines['bottom'].set_visible(True)
ax[0, 2].spines['left'].set_visible(True)


#ETTm2

ettm2= [0,grouped["mse"][3],grouped_norm["mse"][3],grouped_norm_dec["mse"][3]]
ettm2n= ["Vanilla","Enc","Norm","Decom"]
ettm2l = [0,np.round(grouped["mse"][3],2),np.round(grouped_norm["mse"][3],2),np.round(grouped_norm_dec["mse"][3],2)]
ax[1, 0].bar(ettm2n, ettm2,color  =bar_colors,edgecolor="k",label=ettm2l)
ax[1, 0].set_ylabel('MSE Values')
ax[1, 0].set_xlabel('Data Processing')
ax[1, 0].grid(False)
ax[1, 0].set_title('MSE from ETTm2')
ax[1, 0].legend(title='MSE Values')
ax[1, 0].grid(False)
ax[1, 0].spines['top'].set_visible(True)
ax[1, 0].spines['right'].set_visible(True)
ax[1, 0].spines['bottom'].set_visible(True)
ax[1, 0].spines['left'].set_visible(True)
ax[1, 0].spines['top'].set_visible(True)
ax[1, 0].spines['right'].set_visible(True)
ax[1, 0].spines['bottom'].set_visible(True)
ax[1, 0].spines['left'].set_visible(True)



#Heart

Heart= [0,grouped["mse"][4],grouped_norm["mse"][4],grouped_norm_dec["mse"][4]]
Heartn= ["Vanilla","Enc","Norm","Decom"]
hl= [0,np.round(grouped["mse"][4],2),np.round(grouped_norm["mse"][4],2),np.round(grouped_norm_dec["mse"][4],2)]
ax[1, 1].bar(Heartn, Heart,color  =bar_colors,edgecolor="k",label=hl )
ax[1, 1].grid(False)
ax[1, 1].set_ylabel('MSE Values')
ax[1, 1].set_xlabel('Data Processing')

ax[1, 1].set_title('MSE from Heart')
ax[1, 1].legend(title='MSE Values')

ax[1, 1].spines['top'].set_visible(True)
ax[1, 1].spines['right'].set_visible(True)
ax[1, 1].spines['bottom'].set_visible(True)
ax[1, 1].spines['left'].set_visible(True)


#Suns
Suns= [0,grouped["mse"][5],grouped_norm["mse"][5],grouped_norm_dec["mse"][5]]
Sunsn= ["Vanilla","Enc","Norm","Decom"]
sl = [0,np.round(grouped["mse"][5],2),np.round(grouped_norm["mse"][5],2),np.round(grouped_norm_dec["mse"][5],2)]
ax[1, 2].bar(Sunsn, Suns,color  =bar_colors,edgecolor="k",label = sl )
ax[1, 2].set_ylabel('MSE Values')
ax[1, 2].set_xlabel('Data Processing')
ax[1, 2].grid(False)
ax[1, 2].set_title('MSE from Sun')
ax[1, 2].legend(title='MSE Values')


ax[1, 2].spines['top'].set_visible(True)
ax[1, 2].spines['right'].set_visible(True)
ax[1, 2].spines['bottom'].set_visible(True)
ax[1, 2].spines['left'].set_visible(True)


#plt.show()
fig.savefig('BarPltMSE_Exp1_10.png')


In [ ]:
bar_colors = ['tab:red', 'tab:blue', 'tab:orange','tab:green']
#bar_labels = [str(grouped["mse"][0]), str(grouped["mse"][1]), str(grouped["mse"][2]), str(grouped["mse"][3]),str(grouped["mse"][4]),str(grouped["mse"][5])]

fig, ax = plt.subplots(2,3) 
fig.set_figheight(15)
fig.set_figwidth(15)


#ETTh1
etth1= [0,grouped["mape"][0],grouped_norm["mape"][0],grouped_norm_dec["mape"][0]]
etth1l = [0,np.round(grouped["mape"][0],2),np.round(grouped_norm["mape"][0],2),np.round(grouped_norm_dec["mape"][0],2)]
etth1n= ["Vanilla","Enc","Norm","Decom"]
ax[0, 0].bar(etth1n, etth1,color  =bar_colors, edgecolor="k" ,label= etth1l)
ax[0, 0].grid(False)
ax[0, 0].set_ylabel('mape Values')
ax[0, 0].set_xlabel('Data Processing')
ax[0, 0].set_ylim([0, 50])
ax[0, 0].set_title('mape from ETTh1')
ax[0, 0].legend(title='mape Values')

ax[0, 0].spines['top'].set_visible(True)
ax[0, 0].spines['right'].set_visible(True)
ax[0, 0].spines['bottom'].set_visible(True)
ax[0, 0].spines['left'].set_visible(True)




#ETTh2
etth2= [0,grouped["mape"][1],grouped_norm["mape"][1],grouped_norm_dec["mape"][1]]
etth2l = [0,np.round(grouped["mape"][1],2),np.round(grouped_norm["mape"][1],2),np.round(grouped_norm_dec["mape"][1],2)]
etth2n= ["Vanilla","Enc","Norm","Decom"]
ax[0, 1].bar(etth2n, etth2,color  =bar_colors,edgecolor="k",label= etth2l)
ax[0, 1].set_ylim([0, 50])
ax[0, 1].set_ylabel('mape Values')
ax[0, 1].set_xlabel('Data Processing')
ax[0, 1].grid(False)
ax[0, 1].set_title('mape from ETTh2')
ax[0, 1].legend(title='mape Values')


ax[0, 1].spines['top'].set_visible(True)
ax[0, 1].spines['right'].set_visible(True)
ax[0, 1].spines['bottom'].set_visible(True)
ax[0, 1].spines['left'].set_visible(True)



#ETTm1
ettm1= [0,grouped["mape"][2],grouped_norm["mape"][2],grouped_norm_dec["mape"][2]]
ettm1n= ["Vanilla","Enc","Norm","Decom"]
ettm1l = [0,np.round(grouped["mape"][2],2),np.round(grouped_norm["mape"][2],2),np.round(grouped_norm_dec["mape"][2],2)]
ax[0, 2].bar(ettm1n, ettm1,color  =bar_colors ,edgecolor="k",label=ettm1l)
ax[0, 2].set_ylabel('mape Values')
ax[0, 2].set_xlabel('Data Processing')
ax[0, 2].set_ylim([0, 50])
ax[0, 2].set_title('mape from ETTm1')
ax[0, 2].legend(title='mape Values')
ax[0, 2].grid(False)

ax[0, 2].spines['top'].set_visible(True)
ax[0, 2].spines['right'].set_visible(True)
ax[0, 2].spines['bottom'].set_visible(True)
ax[0, 2].spines['left'].set_visible(True)



#ETTm2

ettm2= [0,grouped["mape"][3],grouped_norm["mape"][3],grouped_norm_dec["mape"][3]]
ettm2n= ["Vanilla","Enc","Norm","Decom"]
ettm2l = [0,np.round(grouped["mape"][3],2),np.round(grouped_norm["mape"][3],2),np.round(grouped_norm_dec["mape"][3],2)]
ax[1, 0].bar(ettm2n, ettm2,color  =bar_colors,edgecolor="k",label=ettm2l)
ax[1, 0].set_ylabel('mape Values')
ax[1, 0].set_xlabel('Data Processing')
ax[1, 0].set_ylim([0, 50])
ax[1, 0].set_title('mape from ETTm2')
ax[1, 0].legend(title='mape Values')
ax[1, 0].grid(False)
ax[1, 0].spines['top'].set_visible(True)
ax[1, 0].spines['right'].set_visible(True)
ax[1, 0].spines['bottom'].set_visible(True)
ax[1, 0].spines['left'].set_visible(True)



#Heart

Heart= [0,grouped["mape"][4],grouped_norm["mape"][4],grouped_norm_dec["mape"][4]]
Heartn= ["Vanilla","Enc","Norm","Decom"]
hl= [0,np.round(grouped["mape"][4],2),np.round(grouped_norm["mape"][4],2),np.round(grouped_norm_dec["mape"][4],2)]
ax[1, 1].bar(Heartn, Heart,color  =bar_colors,edgecolor="k",label=hl )
ax[1, 1].set_ylim([0, 50])
ax[1, 1].set_ylabel('mape Values')
ax[1, 1].set_xlabel('Data Processing')
ax[1, 1].grid(False)
ax[1, 1].set_title('mape from Heart')
ax[1, 1].legend(title='mape Values')

ax[1, 1].spines['top'].set_visible(True)
ax[1, 1].spines['right'].set_visible(True)
ax[1, 1].spines['bottom'].set_visible(True)
ax[1, 1].spines['left'].set_visible(True)



#Suns
Suns= [0,grouped["mape"][5],grouped_norm["mape"][5],grouped_norm_dec["mape"][5]]
Sunsn= ["Vanilla","Enc","Norm","Decom"]
sl = [0,np.round(grouped["mape"][5],2),np.round(grouped_norm["mape"][5],2),np.round(grouped_norm_dec["mape"][5],2)]
ax[1, 2].bar(Sunsn, Suns,color  =bar_colors,edgecolor="k",label = sl )
ax[1, 2].set_ylabel('mape Values')
ax[1, 2].set_xlabel('Data Processing')
ax[1, 2].set_ylim([0, 50])
ax[1, 2].set_title('mape from Sun')
ax[1, 2].legend(title='mape Values')
ax[1, 2].grid(False)

ax[1, 2].spines['top'].set_visible(True)
ax[1, 2].spines['right'].set_visible(True)
ax[1, 2].spines['bottom'].set_visible(True)
ax[1, 2].spines['left'].set_visible(True)

#plt.show()
fig.savefig('BarPltmape_Exp1_10.png')


In [ ]:
bar_colors = ['tab:red', 'tab:blue', 'tab:orange','tab:green']
#bar_labels = [str(grouped["mse"][0]), str(grouped["mse"][1]), str(grouped["mse"][2]), str(grouped["mse"][3]),str(grouped["mse"][4]),str(grouped["mse"][5])]

fig, ax = plt.subplots(2,3) 
fig.set_figheight(15)
fig.set_figwidth(15)


#ETTh1
etth1= [0,grouped["mae"][0],grouped_norm["mae"][0],grouped_norm_dec["mae"][0]]
etth1l = [0,np.round(grouped["mae"][0],2),np.round(grouped_norm["mae"][0],2),np.round(grouped_norm_dec["mae"][0],2)]
etth1n= ["Vanilla","Enc","Norm","Decom"]
ax[0, 0].bar(etth1n, etth1,color  =bar_colors, edgecolor="k" ,label= etth1l)
ax[0, 0].grid(False)
ax[0, 0].set_ylabel('mae Values')
ax[0, 0].set_xlabel('Data Processing')
ax[0, 0].set_ylim([0, 50])
ax[0, 0].set_title('mae from ETTh1')
ax[0, 0].legend(title='mae Values')

ax[0, 0].spines['top'].set_visible(True)
ax[0, 0].spines['right'].set_visible(True)
ax[0, 0].spines['bottom'].set_visible(True)
ax[0, 0].spines['left'].set_visible(True)



#ETTh2
etth2= [0,grouped["mae"][1],grouped_norm["mae"][1],grouped_norm_dec["mae"][1]]
etth2l = [0,np.round(grouped["mae"][1],2),np.round(grouped_norm["mae"][1],2),np.round(grouped_norm_dec["mae"][1],2)]
etth2n= ["Vanilla","Enc","Norm","Decom"]
ax[0, 1].bar(etth2n, etth2,color  =bar_colors,edgecolor="k",label= etth2l)
ax[0, 1].set_ylim([0, 50])
ax[0, 1].set_ylabel('mae Values')
ax[0, 1].set_xlabel('Data Processing')
ax[0, 1].grid(False)
ax[0, 1].set_title('mae from ETTh2')
ax[0, 1].legend(title='mae Values')
ax[0, 1].spines['top'].set_visible(True)
ax[0, 1].spines['right'].set_visible(True)
ax[0, 1].spines['bottom'].set_visible(True)
ax[0, 1].spines['left'].set_visible(True)



#ETTm1
ettm1= [0,grouped["mae"][2],grouped_norm["mae"][2],grouped_norm_dec["mae"][2]]
ettm1n= ["Vanilla","Enc","Norm","Decom"]
ettm1l = [0,np.round(grouped["mae"][2],2),np.round(grouped_norm["mae"][2],2),np.round(grouped_norm_dec["mae"][2],2)]
ax[0, 2].bar(ettm1n, ettm1,color  =bar_colors ,edgecolor="k",label=ettm1l)
ax[0, 2].set_ylabel('mae Values')
ax[0, 2].set_xlabel('Data Processing')
ax[0, 2].set_ylim([0, 50])
ax[0, 2].set_title('mae from ETTm1')
ax[0, 2].legend(title='mae Values')
ax[0, 2].grid(False)
ax[0, 2].spines['top'].set_visible(True)
ax[0, 2].spines['right'].set_visible(True)
ax[0, 2].spines['bottom'].set_visible(True)
ax[0, 2].spines['left'].set_visible(True)


#ETTm2

ettm2= [0,grouped["mae"][3],grouped_norm["mae"][3],grouped_norm_dec["mae"][3]]
ettm2n= ["Vanilla","Enc","Norm","Decom"]
ettm2l = [0,np.round(grouped["mae"][3],2),np.round(grouped_norm["mae"][3],2),np.round(grouped_norm_dec["mae"][3],2)]
ax[1, 0].bar(ettm2n, ettm2,color  =bar_colors,edgecolor="k",label=ettm2l)
ax[1, 0].set_ylabel('mae Values')
ax[1, 0].set_xlabel('Data Processing')
ax[1, 0].set_ylim([0, 50])
ax[1, 0].set_title('mae from ETTm2')
ax[1, 0].legend(title='mae Values')
ax[1, 0].grid(False)

ax[1, 0].spines['top'].set_visible(True)
ax[1, 0].spines['right'].set_visible(True)
ax[1, 0].spines['bottom'].set_visible(True)
ax[1, 0].spines['left'].set_visible(True)




#Heart

Heart= [0,grouped["mae"][4],grouped_norm["mae"][4],grouped_norm_dec["mae"][4]]
Heartn= ["Vanilla","Enc","Norm","Decom"]
hl= [0,np.round(grouped["mae"][4],2),np.round(grouped_norm["mae"][4],2),np.round(grouped_norm_dec["mae"][4],2)]
ax[1, 1].bar(Heartn, Heart,color  =bar_colors,edgecolor="k",label=hl )
ax[1, 1].set_ylim([0, 50])
ax[1, 1].set_ylabel('mae Values')
ax[1, 1].set_xlabel('Data Processing')
ax[1, 1].grid(False)
ax[1, 1].set_title('mae from Heart')
ax[1, 1].legend(title='mae Values')

ax[1, 1].spines['top'].set_visible(True)
ax[1, 1].spines['right'].set_visible(True)
ax[1, 1].spines['bottom'].set_visible(True)
ax[1, 1].spines['left'].set_visible(True)



#Suns
Suns= [0,grouped["mae"][5],grouped_norm["mae"][5],grouped_norm_dec["mae"][5]]
Sunsn= ["Vanilla","Enc","Norm","Decom"]
sl = [0,np.round(grouped["mae"][5],2),np.round(grouped_norm["mae"][5],2),np.round(grouped_norm_dec["mae"][5],2)]
ax[1, 2].bar(Sunsn, Suns,color  =bar_colors,edgecolor="k",label = sl )
ax[1, 2].set_ylabel('mae Values')
ax[1, 2].set_xlabel('Data Processing')
ax[1, 2].set_ylim([0, 50])
ax[1, 2].set_title('mae from Sun')
ax[1, 2].legend(title='mae Values')
ax[1, 2].grid(False)

ax[1, 2].spines['top'].set_visible(True)
ax[1, 2].spines['right'].set_visible(True)
ax[1, 2].spines['bottom'].set_visible(True)
ax[1, 2].spines['left'].set_visible(True)


#plt.show()
fig.savefig('BarPltmae_Exp1_10.png')


In [ ]:
def plot_time_series1(gt, series1,series2,series3,empty_array,input,path,name):
    """
    Plot two time series arrays.
    
    Args:
    series1 (list or numpy array): First time series data.
    series2 (list or numpy array): Second time series data.
    """
    gt = np.append(input,gt)
    series1 = np.append(empty_array,series1)
    series2 = np.append(empty_array,series2)
    series3 = np.append(empty_array,series3)
    
    # Generate time indices
    time_indices = list(range(1, len(series1) + 1))
   
    # Plot time series
    plt.figure(figsize=(10, 6))
    plt.plot(time_indices, gt, label='gt',color = "k",linestyle='--')
    plt.plot(time_indices, series1, label='Encoding',color = "g")
    plt.plot(time_indices, series2, label='Norm_Encoding',color = "r")
    plt.plot(time_indices, series3, label='Dec_Norm_Encod',color = "orange")
    
    # Add labels and legend
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.title(name)
    plt.legend()
    #plt.text(0.5, 0.5,str(mse), fontsize=14, transform=plt.gcf().transFigure)
    
    # Show plot
    plt.grid(True)

    path= path +"/"+ name + ".png"
    plt.savefig(path)
    #plt.show

In [ ]:
def create_nan_array(length):
    return np.full(length, np.nan)
nan_array = create_nan_array(100)



In [ ]:
for i in range(len(mse_end_table)):

    inv_values_dec = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],mse_combined["answer"][i],mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
    
    inv_values_norm = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],norm_df["answer"][i],mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]
    
    
    plot_time_series1(deserialize_str(enc_dataset["output"][i],settings),enc_df["answer"][i],inv_values_norm,inv_values_dec,nan_array,deserialize_str(enc_dataset["input"][i],settings)[-100:],"ExpPlots_10",norm_df["dataset_name"][i])

In [ ]:
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import numpy as np
 
# Assuming you have three lists of MSE values for each model
mse_model1 = deserialize_str(enc_dataset["output"][i],settings)
mse_model2 = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],mse_combined["answer"][i],mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]

mse_model3 = inv_values_dec = nv_values_norm = [item for sublist in min_max_scale_inv(mse_in_values["values"][i],norm_df["answer"][i],mse_in_values["min"][i],mse_in_values["max"][i]) for item in sublist]  

# Perform ANOVA test
f_statistic, p_value = f_oneway(mse_model1, mse_model2, mse_model3)
print("One-way ANOVA")
print("F-statistic:", f_statistic)
print("p-value:", p_value)

# Perform Tukey's HSD test
mse_values = np.concatenate([mse_model1, mse_model2, mse_model3])
model_labels = ['Model 1'] * len(mse_model1) + ['Model 2'] * len(mse_model2) + ['Model 3'] * len(mse_model3)
tukey_results = pairwise_tukeyhsd(mse_values, model_labels)
print("\nTukey's HSD")
print(tukey_results)


In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px
X= np.array([mse_combined["answer"][0],mse_combined["answer"][1]])

X_embedded = TSNE(n_components=2, learning_rate='auto',
             init='random', perplexity=1).fit_transform(X)
X_embedded.shape

In [ ]:
import plotly.express as px
X_res= np.array([deserialize_str(dec_dataset["remain"][4],settings),deserialize_str(dec_dataset["remain"][5],settings)])
X_res=np.reshape(X_res,(783 ,-2))
tsne = TSNE(n_components=2, random_state=42,perplexity=30,n_iter=1000)
X_tsne_res = tsne.fit_transform(X_res)
tsne.kl_divergence_

In [ ]:
X_seas= np.array([deserialize_str(dec_dataset["season"][4],settings),deserialize_str(dec_dataset["season"][5],settings)])
X_seas=np.reshape(X_seas,(783 ,-2))
tsne = TSNE(n_components=2, random_state=42,perplexity=30,n_iter=1000)
X_tsne_seas = tsne.fit_transform(X_seas)

In [ ]:
X_trend= np.array([deserialize_str(dec_dataset["trend"][4],settings),deserialize_str(dec_dataset["trend"][5],settings)])
X_trend=np.reshape(X_trend,(783 ,-2))
tsne = TSNE(n_components=2, random_state=42,perplexity=30,n_iter=1000)
X_tsne_trend = tsne.fit_transform(X_trend)

In [ ]:
X_val = np.append(X_tsne_res[:, 0],X_tsne_seas[:, 0])
X_val = np.append(X_val,X_tsne_trend[:, 0])
y_val = np.append(X_tsne_res[:, 1],X_tsne_seas[:, 1])
y_val= np.append(y_val,X_tsne_trend[:, 1])

In [ ]:
dec_dataset

In [ ]:
fig = px.scatter(x=X_val, y=y_val,color= y)
fig.update_layout(
    title="t-SNE visualization of Custom Classification dataset",
    xaxis_title="First t-SNE",
    yaxis_title="Second t-SNE",
)
fig.show()

In [ ]:
len(deserialize_str(dec_dataset["trend"][0],settings))

In [ ]:
res=np.full(783, 0)
season= np.full(783, 1)
trend = np.full(783, 2)

In [ ]:
y=np.append(res,season)
y= np.append(y,trend)

In [ ]:
len(y)


In [ ]:
np.max(trend1)

In [ ]:
sesaon1 = (deserialize_str(dec_dataset["season"][0],settings))
np.min(sesaon1)

In [ ]:
np.max(sesaon1)

In [ ]:
X_tsne_res

In [ ]:
X_tsne_seas